# Context Augmented Generation (CAG) with LlamaIndex
**Concept:** We do not index the data into a vector database. We load the **entire** text into the LLM's 'working memory' (Context Window).

**Best For:** Analyzing specific files (Contracts, Memos) where you need the model to see the "whole picture".

**Technique:** We use `Document` objects and feed them directly into the prompt.

In [1]:
from llama_index.core import Document, Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage

# 1. Setup the LLM
Settings.llm = Ollama(model="qwen3:0.6b", request_timeout=60.0)

# 2. Define the "Knowledge" (The Context)
# In a real scenario, this would be loaded from a PDF or Text file.
policy_text = """
MUNICIPALITY OF AVEIRO - REMOTE WORK POLICY 2024

Section 1: Eligibility
Employees are eligible for remote work if they have completed their probation period (6 months).
Exceptions are made for employees with health conditions, verified by a medical officer.

Section 2: Equipment
The municipality will provide one laptop and one monitor.
Internet costs are reimbursed at a flat rate of €20/month.

Section 3: Core Hours
Regardless of location, employees must be available between 10:00 AM and 3:00 PM.
"""

document = Document(text=policy_text)

In [2]:
# 3. The CAG Execution
# We manually inject the document text into the system prompt.

def ask_cag(question):
    # Create a message history where the first message contains the FULL context
    messages = [
        ChatMessage(
            role="system",
            content=f"""You are an HR Assistant.
            Answer strictly based on the following policy:\n\n{document.text}
            """
        ),
        ChatMessage(role="user", content=question),
    ]

    resp = Settings.llm.chat(messages)
    return resp.message.content

# 4. Test
print("--- Query 1 ---")
print(ask_cag("I have been working here for 2 months. Can I work from home?"))

print("\n--- Query 2 ---")
print(ask_cag("Does the city pay for my home internet?"))

--- Query 1 ---
Based on the provided policy, you **cannot work from home** at this time. You have only been working here for 2 months, which is less than the 6-month probation period required to qualify for remote work. The municipality will provide a laptop and monitor, but you must still complete your probation period (6 months) before being eligible for remote work. Additionally, you must be available between 10:00 AM and 3:00 PM, which you already meet. If you're eligible for remote work after completing your probation, you may be able to work from home. Let me know if you'd like further details!

--- Query 2 ---
Yes, the municipality reimburses the flat rate of €20/month for internet costs. This means your home internet is covered at the specified monthly rate. Let me know if you need further details!
